# Kepler Framework, Emulate v9

In [2]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [4]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [5]:
tpath_data17 = 'output/emulation/data17_table_v9.h5'
tpath_data18 = 'output/emulation/data18_table_v9.h5'
tpath_mc16   = 'output/emulation/mc16_table_v9.h5'

## Setup Chains:

In [6]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v9_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v9_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v9_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v9_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v9" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v9_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v9_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v9_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v9_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v9_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [7]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [8]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:35<00:00, 23.77s/it]


In [9]:
data17_df.shape

(43311283, 111)

### Validate:

In [10]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v9.root')

2022-03-09 18:36:00,902 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v9.root


In [11]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.55s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.4993  | 29964235/32047563 |
|   L2   | 93.2617  | 29888090/32047563 |
| EFCalo | 91.9508  | 29468006/32047563 |
|  HLT   | 90.0283  | 28851868/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.40s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2570  | 21458184/22765602 |
|   L2   | 94.1426  | 21432137/22765602 |
| EFCalo | 93.2183  | 21221698/22765602 |
|  HLT   | 87.0467  | 19816696/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.3732  | 1331857/1367786 |
|   L2   | 97.2831  | 1330624/1367786 |
| EFCalo | 95.1930  | 1302037/1367786 |
|  HLT   | 91.9069  | 1257090/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 19.96it/s]


HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.0592  | 59366/60541  |
|   L2   | 97.9898  | 59324/60541  |
| EFCalo | 96.6700  | 58525/60541  |
|  HLT   | 94.7127  | 57340/60541  |
+--------+----------+--------------+


In [12]:
eff_data17_Zee.save()

In [13]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v9.root')

2022-03-09 18:40:16,284 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v9.root


In [14]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.39s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  1.9152  | 198223/10349992  |
|   L2   |  1.8345  | 189869/10349992  |
| EFCalo |  1.6375  | 169476/10349992  |
|  HLT   |  0.1436  |  14863/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.97s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  1.1724  |  108494/9254300 |
|   L2   |  1.1142  |  103107/9254300 |
| EFCalo |  0.9351  |  86540/9254300  |
|  HLT   |  0.0020  |   184/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.35s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  1.5721  |  123447/7852181 |
|   L2   |  1.4934  |  117261/7852181 |
| EFCalo |  1.2574  |  98730/7852181  |
|  HLT   |  0.0035  |   271/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.49s/it]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.1381  |  120463/5634041 |
|   L2   |  2.0201  |  113816/5634041 |
| EFCalo |  1.7374  |  97888/5634041  |
|  HLT   |  0.0402  |   2265/5634041  |
+--------+----------+-----------------+


In [15]:
eff_data17_JF17.save()

In [16]:
del data17_df

## Emulate 2018:

In [17]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [18]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:36<00:00, 24.10s/it]


In [19]:
data18_df.shape

(41873973, 111)

In [20]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v9.root')

2022-03-09 18:45:37,948 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v9.root


In [21]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.60s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5427  | 30514089/32620495 |
|   L2   | 93.3121  | 30438857/32620495 |
| EFCalo | 91.9784  | 30003816/32620495 |
|  HLT   | 90.5428  | 29535495/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.53s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2189  | 21966035/23313826 |
|   L2   | 94.1238  | 21943853/23313826 |
| EFCalo | 93.3247  | 21757557/23313826 |
|  HLT   | 87.6113  | 20425556/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3526  | 1345097/1381675 |
|   L2   | 97.2643  | 1343876/1381675 |
| EFCalo | 95.1778  | 1315048/1381675 |
|  HLT   | 92.1894  | 1273758/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.31it/s]


HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.1619  | 59279/60389  |
|   L2   | 98.0609  | 59218/60389  |
| EFCalo | 96.8753  | 58502/60389  |
|  HLT   | 95.0355  | 57391/60389  |
+--------+----------+--------------+


In [22]:
eff_data18_Zee.save()

In [23]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v9.root')

2022-03-09 18:49:58,704 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v9.root


In [24]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.58s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.2917  |  194042/8467098 |
|   L2   |  2.2187  |  187860/8467098 |
| EFCalo |  2.0274  |  171663/8467098 |
|  HLT   |  0.6299  |  53338/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.21s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.5317  |  116853/7628804 |
|   L2   |  1.4777  |  112729/7628804 |
| EFCalo |  1.3077  |  99761/7628804  |
|  HLT   |  0.0503  |   3839/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.77s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  1.9514  |  128051/6561985 |
|   L2   |  1.8763  |  123122/6561985 |
| EFCalo |  1.6534  |  108494/6561985 |
|  HLT   |  0.0677  |   4441/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.09s/it]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.5505  |  121823/4776518 |
|   L2   |  2.4435  |  116712/4776518 |
| EFCalo |  2.1763  |  103949/4776518 |
|  HLT   |  0.3546  |  16937/4776518  |
+--------+----------+-----------------+


In [25]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [26]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [27]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 29.18it/s]


In [28]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v9.root')

2022-03-09 18:51:15,742 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v9.root


In [29]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 15.47it/s]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 97.2895  | 76488/78619  |
|   L2   | 97.2806  | 76481/78619  |
| EFCalo | 76.3836  | 60052/78619  |
|  HLT   | 75.3380  | 59230/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.39it/s]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 91.1155  | 66610/73105  |
|   L2   | 91.1100  | 66606/73105  |
| EFCalo | 72.0758  | 52691/73105  |
|  HLT   | 69.3181  | 50675/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.89it/s]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 96.1008  | 71055/73938  |
|   L2   | 96.0954  | 71051/73938  |
| EFCalo | 75.0818  | 55514/73938  |
|  HLT   | 73.2343  | 54148/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.12it/s]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 97.1878  | 66044/67955  |
|   L2   | 97.1834  | 66041/67955  |
| EFCalo | 75.6456  | 51405/67955  |
|  HLT   | 74.2624  | 50465/67955  |
+--------+----------+--------------+
